In [1]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

0.4.0
True


In [2]:
!pip3 install Pillow==4.0.0
!pip3 install PIL
!pip3 install image

  Using cached https://files.pythonhosted.org/packages/37/e8/b3fbf87b0188d22246678f8cd61e23e31caa1769ebc06f1664e2e5fe8a17/Pillow-4.0.0-cp36-cp36m-manylinux1_x86_64.whl
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.2.0
    Uninstalling Pillow-5.2.0:
      Successfully uninstalled Pillow-5.2.0
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL


In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets
import torch.nn as nn
import torch.nn.functional as F

In [0]:
trans = transforms.Compose([transforms.ToTensor()])

In [0]:
trainset = torchvision.datasets.MNIST(root='./data',train=True,download=True,transform=trans)

In [0]:
testset = torchvision.datasets.MNIST(root='./data',train=False,download=True,transform=trans)

In [0]:
train_loader = torch.utils.data.DataLoader(
                 dataset=trainset,
                 shuffle=True)

In [0]:
test_loader = torch.utils.data.DataLoader(dataset=testset,shuffle=True)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.fc1 = nn.Linear(1024, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = F.dropout(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 1024)
        x = F.relu(self.fc1(x))
        return x

In [0]:
import torch.optim as optim

In [0]:
loss_func = nn.CrossEntropyLoss()

In [0]:
model = Net()

In [0]:
model = model.cuda()

In [0]:
optimizer = optim.SGD(model.parameters(),lr=0.001,momentum=0.95)

In [0]:
from torch.autograd import Variable

In [16]:
for epoch in range(2):
  for i,(x,y) in enumerate(train_loader):
    
    x = x.cuda()
    y=y.cuda()
    inputs, labels = Variable(x),Variable(y)
    optimizer.zero_grad()
    answer = model(inputs)
    loss = loss_func(answer,labels)
    loss.backward()
    optimizer.step()
    if(i%10000==0):
      print(loss)

tensor(2.3345, device='cuda:0')
tensor(2.3026, device='cuda:0')
tensor(1.00000e-05 *
       7.7248, device='cuda:0')
tensor(0., device='cuda:0')
tensor(1.00000e-06 *
       9.5367, device='cuda:0')
tensor(2.3026, device='cuda:0')
tensor(1.00000e-05 *
       3.2425, device='cuda:0')
tensor(0., device='cuda:0')
tensor(1.00000e-06 *
       3.8147, device='cuda:0')
tensor(2.3026, device='cuda:0')
tensor(2.3026, device='cuda:0')
tensor(1.00000e-06 *
       1.9073, device='cuda:0')


In [0]:
correct=0
total=len(test_loader)

In [18]:
total

10000

In [0]:
with torch.no_grad():
    for image,label in test_loader:
        image = image.cuda()
        label = label.cuda()
        output = model(image)
        pred = output.max(1,True)[1]
        if(pred==label):
          correct+=1

In [22]:
correct



8916

In [23]:
print('Accuracy:'+str(correct/total))

Accuracy:0.8916
